# Train on arctic-sea-ice sentinel images (pre and post melt)

### Experiment setup
- sen2 arctic sea ice: pre-melt image and post-melt image
- binary masks 1024x1024 from SWIR band
- train on Train

### Display config

In [1]:
%%bash
cd ../..
. scripts/config/arctic_sea_ice.sh
echo 'bucket_path: '${bucket_path}
echo 'seg_model_path: '${seg_model_path} 
echo 'seg_model_name: '${seg_model_name} 

echo 'root_dir: '${root_dir} 
echo 'dataroot: '${dataroot}
echo 'model_name: '${model_name}
echo 'ckpt_dir: '${ckpt_dir}
echo 'temp_dir: '${temp_dir}
echo 'results_dir: '${results_dir}

bucket_path: eie-arctic-sea-ice/
seg_model_path: pretrained/pix2pix_seg/arctic_sea_ice/
seg_model_name: latest_net_G.pth
root_dir: /home/jupyter/eie_vision/
dataroot: data/eie-arctic-sea-ice-test/
model_name: conditional_binary_scratch_arctic_sea_ice_test/
ckpt_dir: temp/checkpoint/Pix2pixHD/
temp_dir: temp/Pix2pixHD/
results_dir: results/Pix2pixHD/


### Load data into instance

In [1]:
%%bash
# Load data from bucket into instance (in terminal)
# TODO also load test.
cd ../..; . scripts/config/arctic_sea_ice.sh 

split_src=('Train_A/' 'Train_B/' 'Train_B_SWIR/')
split_dst=('train_A/' 'train_B/' 'train_mask/')
for ((i=0;i<${#split_src[@]};++i)); do
    mkdir -p ${root_dir}data/${bucket_path}${split_dst[i]}
    gsutil -m cp -r gs://${bucket_path}${split_src[i]}* ${root_dir}data/${bucket_path}${split_dst[i]}
done

SyntaxError: invalid syntax (<ipython-input-1-075d10991d5f>, line 3)

In [2]:
# Check that the folder pairs _A, _B, _mask have the same number of filenames
!ls /home/jupyter/eie_vision/data/eie-arctic-sea-ice-uniq/train_A | wc -l ;\
ls /home/jupyter/eie_vision/data/eie-arctic-sea-ice-uniq/train_B | wc -l ;\
ls /home/jupyter/eie_vision/data/eie-arctic-sea-ice-uniq/train_mask | wc -l

15000
15000
15000


### Train pix2pixHD

In [3]:
# Remove model and jupyter checkpoints
!cd /home/jupyter/eie_vision; . scripts/config/arctic_sea_ice.sh ;\
rm -r ${root_dir}${ckpt_dir}${model_name} ;\
rm -r ${root_dir}${dataroot}*/.ipynb_checkpoints

In [4]:
!cd /home/jupyter/eie_vision; . scripts/config/arctic_sea_ice.sh ;\
python src/models/Pix2pixHD/train.py \
--no_lpips_loss --save_epoch_freq 1 --niter 50 --niter_decay 50 \
--checkpoints_dir ${ckpt_dir} --name ${model_name} --dataroot ${dataroot} \
--dataset_mode physics_aligned_bin --no_instance \
--label_nc 0 --input_nc 4  --batchSize 8 --gpu_ids 0,1,2,3,4,5,6,7

^C


In [6]:
# Train pix2pixHD model
# !cd /home/jupyter/eie_vision; . scripts/config/arctic_sea_ice.sh ;\
# python src/models/Pix2pixHD/train.py \
# --continue_train  --no_lpips_loss --save_epoch_freq 15 --niter 100 \
# --niter_decay 100 --checkpoints_dir ${ckpt_dir} --name ${model_name} \
# --dataroot ${dataroot} --dataset_mode physics_aligned_bin --no_instance \
# --label_nc 0 --input_nc 4  --batchSize 8 --gpu_ids 0,1,2,3,4,5,6,7 \
# --tf_log
%%bash

!cd /home/jupyter/eie_vision;
python src/models/Pix2pixHD/train.py --no_lpips_loss --save_epoch_freq 5 --niter 50 --niter_decay 50 --checkpoints_dir  temp/checkpoint/Pix2pixHD/ \
--name "conditional_binary_scratch_arctic_sea_ice_uniq/" --dataroot "data/eie-arctic-sea-ice-uniq/" --dataset_mode physics_aligned_bin --no_instance \
--label_nc 0 --input_nc 4  --batchSize 8 --gpu_ids 0,1,2,3,4,5,6,7

# Description of flags:
# --continue_train continues training from checkpoint if available
# --no_lpips_loss trains w/o the non-working lpips loss
# --save_epoch_freq is frequency of creating checkpoints
# --niter is the maximum number of epochs
# --niter_decay is the number of iterations until a decay of learning rate starts
# --dataset_mode physics_aligned_bin sets training to incorporate the mask input
# --no_instance
# --label_nc is number of channels in label, here 0, so we dont have any.
# --input_nc is number of input channels, e.g., 4 for RGB + mask
# --checkpoints_dir is directory of model checkpoints
# --batchSize 8 is batch size -- Let's experiment with this. Maybe multiple images per GPU
# --gpu_ids 0,1,2,3,4,5,6,7 are the gpu IDS

SyntaxError: invalid syntax (<ipython-input-6-28be66d25a68>, line 12)

### Generate predictions over the test set. 

In [ ]:
# TODO: create test_A, test_B, and test_mask folder in ${dataroot}

In [25]:
!cd /home/jupyter/eie_vision; . scripts/config/arctic_sea_ice.sh ;\
python src/models/Pix2pixHD/test.py \
--checkpoints_dir ${ckpt_dir} --name ${model_name} \
--phase test --how_many 2000 \
--dataroot ${dataroot} --dataset_mode physics_aligned_bin --no_instance \
--label_nc 0 --input_nc 4 --batchSize 8 --gpu_ids 0,1,2,3,4,5,6,7 \
--results_dir ${temp_dir}

------------ Options -------------
aspect_ratio: 1.0
augmentation_aggressive: False
batchSize: 8
checkpoints_dir: temp/checkpoint/Pix2pixHD/
cluster_path: features_clustered_010.npy
data_type: 32
dataroot: data/eie_floods/naip/xbd/images
dataset_mode: physics_aligned_bin
display_winsize: 512
engine: None
export_onnx: None
feat_num: 3
fineSize: 512
fp16: False
gpu_ids: [0, 1, 2, 3, 4, 5, 6, 7]
how_many: 2000
input_nc: 4
instance_feat: False
isTrain: False
label_feat: False
label_nc: 0
loadSize: 1024
load_features: False
local_rank: 0
max_dataset_size: inf
model: pix2pixHD
nThreads: 8
n_blocks_global: 9
n_blocks_local: 3
n_clusters: 10
n_downsample_E: 4
n_downsample_global: 4
n_local_enhancers: 1
name: conditional_binary_scratch_naip
nef: 16
netG: global
ngf: 64
niter_fix_global: 0
no_flip: False
no_instance: True
norm: instance
normD: spectral
ntest: inf
onnx: None
output_nc: 3
phase: test
resize_or_crop: scale_width
results_dir: temp/Pix2pixHD/
serial_batches: False
slosh_inference: Fa

### Create segmentation masks for generated imagery and move imagery to results/ dir
We use the ./temp directly to work with the imagery and then carefully select imagery for the ./results directory, which is synced with the buckets.

In [ ]:
%%bash
# Remove unnecessary outputs and rename generated imagery 
cd ../..; . scripts/config/arctic_sea_ice.sh

rm -r ${root_dir}${temp_dir}${model_name}${temp_ims}/*_input_label.jpg # Remove unnecessary outputs
rename 's/_pre'${im_suffix}'_synthesized_image.jpg/_post'${im_suffix}'.jpg/' ${temp_ims}*

In [1]:
# Download segmentation model and create segmentation masks
!cd /home/jupyter/eie_vision; . scripts/config/arctic_sea_ice.sh ;\
#gsutil -m cp -r gs://${bucket_path}${seg_model_path}${seg_model_name} ${seg_model_path}${seg_model_name} ;\
python src/models/Pix2pix-CycleGAN/test.py \
--load_size 1024 --crop_size 1024 --dataroot ${temp_dir}${model_name}test_latest/to_segment/ --name . \
--checkpoints_dir ${seg_model_path} \
--model pix2pix --direction AtoB \
--results_dir ${temp_dir}${model_name}test_latest/mask/ \
--dataset_mode single --output_nc 1 \
--num_test 1870 --max_dataset_size 1870 --no_flip \
--gpu_ids 0,1,2,3  --batch_size 16

In [ ]:
# Move generated imagery and masks from temp/ to results/ directory
!cd /home/jupyter/eie_vision; sh scripts/move_temp_to_results.sh

### Shutdown instance

In [35]:
!gcloud compute instances stop --quiet --project "fdl-us-2020-eie" --zone "us-west1-b" eie-week1-ml-pix2pixhd

Did you mean zone [us-west1-b] for instance: [eie-week1-ml-pix2pixHD] 
(Y/n)?  ^C


Command killed by keyboard interrupt

